In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from netCDF4 import Dataset
from datetime import datetime, timedelta

The Climate Research Unit gridded Time Series (CRU TS) is a widely used climate dataset with **monthly resolution** on a 0.5° latitude by 0.5° longitude grid over all land domains of the world except Antarctica. The dataset is introduced in this [paper](https://doi.org/10.1038/s41597-020-0453-3).

![](cru-ts-vars.png)

In [2]:
cru_ts_tmp = Dataset('../../data/CRU-TS/cru_ts4.05.1901.2020.tmp.dat.nc')
cru_ts_pre = Dataset('../../data/CRU-TS/cru_ts4.05.1901.2020.pre.dat.nc')

In [3]:
# import time var & convert into datetime object
time = cru_ts_tmp.variables['time'][:]

dtime = []
start = datetime(1900, 1, 1)
for t in time:
    delta = timedelta(days=int(t))
    dtime.append(start + delta)

In [4]:
# get the data
# NOTE: taking every s-th coord due to computing power limitations
s = 10
pre = cru_ts_pre.variables['pre'][:,::s,::s]
tmp = cru_ts_tmp.variables['tmp'][:,::s,::s] 
#pre = ppre * 100    # convert to cm
#tmp = ttmp - 273.15 # convert to °C
# leads to overflow error (array's too big for copying)
lon = cru_ts_tmp.variables['lon'][::s]
lat = cru_ts_tmp.variables['lat'][::s]
print(tmp.shape)
print(pre.shape)
print(lon.shape)
print(lat.shape)

(1440, 36, 72)
(1440, 36, 72)
(72,)
(36,)


In [5]:
# select time frame of interest (i. e., the anomaly maps are created for these years)
# NOTE: for this to work flawless, do not change month and day values!
first_year = 1998
final_year = 2020

anom_start = datetime(first_year, 1, 16)
anom_end = datetime(final_year, 12, 16)

ianom_start = dtime.index(anom_start)
ianom_end = dtime.index(anom_end) + 1 # indexing in python does not include stop value 

print(ianom_start, ianom_end)

1164 1440


### Precipitation

Accumulated liquid and frozen water, including rain and snow, that falls to the Earth's surface. It is the sum of large-scale precipitation (that precipitation which is generated by large-scale weather patterns, such as troughs and cold fronts) and convective precipitation (generated by convection which occurs when air at lower levels in the atmosphere is warmer and less dense than the air above, so it rises). Precipitation variables do not include fog, dew or the precipitation that evaporates in the atmosphere before it lands at the surface of the Earth. This variable is accumulated from the beginning of the forecast time to the end of the forecast step. **The units of precipitation are depth in metres. It is the depth the water would have if it were spread evenly over the grid box.** Care should be taken when comparing model variables with observations, because observations are often local to a particular point in space and time, rather than representing averages over a model grid box and model time step.

### Temperature

Temperature of air at 2m above the surface of land, sea or in-land waters. 2m temperature is calculated by interpolating between the lowest model level and the Earth's surface, taking account of the atmospheric conditions. **Temperature measured in kelvin can be converted to degrees Celsius (°C) by subtracting 273.15**.

In [6]:
# The data is defined in lat/lon coordinate system, so PlateCarree() is the
# appropriate transformation choice.
# (https://scitools.org.uk/cartopy/docs/latest/tutorials/understanding_transform.html)
data_crs = ccrs.PlateCarree()

def plot_maps(var, kind, cmap, vmin, vmax, unit):
    number_of_plots = 1#(final_year + 1) - first_year
    idummy = 0
    # collection of subplots for every year of time frame of interest
    for iplot in range(number_of_plots):
        current_year = dtime[ianom_start+idummy].strftime('%Y')
        print(f"{kind}: plotting #{iplot+1} of {number_of_plots}...")
        # as there are lots of figures and we save them directly anyways, let's
        # clear them from memory:
        # https://stackoverflow.com/questions/28757348/how-to-clear-memory-completely-of-all-matplotlib-plots#55834853
        fig, axs = plt.subplots(
            nrows=3, ncols=4, figsize=(16,9),
            subplot_kw={'projection':ccrs.Miller()}, # determine map projection
            num=1, clear=True # use same figure, but cleared (prevents memory overflow)
        )
        fig.suptitle(f"{current_year}: monthly {kind} mean", fontsize=20)
        # plotting individual month
        for ax in axs.flat:
            ax.set_extent([-25, 70, 5, 70])
            ax.set_title(dtime[ianom_start+idummy].strftime('%b'))
            pcm = ax.pcolormesh(
                lon, lat, var[idummy],
                transform=data_crs, cmap=cmap,
                rasterized=True, shading='nearest', # these settings are imprtant
                                                    # for properly rendering pdf
                vmin=vmin, vmax=vmax
            )
            ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
            idummy += 1
        cbar = plt.colorbar(pcm, ax=axs[:,:], shrink=0.9)
        cbar.set_label(f'{kind} [{unit}]', fontsize=15)
        fig.savefig(
            f'../figs/kaliningrad/maps/{kind}-{current_year}.pdf', bbox_inches='tight'
        )
        # due to high memory usage nonetheless, added the following, just in case:
        fig.clear()
        plt.close(fig)

# plotting...
plot_maps(tmp, "temperature", "gist_rainbow_r", -12, 41, "°C")
plot_maps(pre, "precipitation", "Blues", 0, 300, "mm")

temperature: plotting #1 of 1...
precipitation: plotting #1 of 1...
